In [1]:
import QDMPy.data_loading as data_loading
import QDMPy.fit_interface as fit_interface
import QDMPy.fit_plots as fit_plots

import matplotlib
import matplotlib.pyplot as plt

In [2]:
# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
%matplotlib widget

# Load the data in

In [3]:
options_dict = {
    "base_dir": "C:\\src\\QDMPY_test_image\\",
    "filepath": "ODMR - Pulsed with IR_83",
    
    "additional_bins": 4,

    "system_name": "Zyla",

    "ROI": "Full", 
    "ROI_centre": [60, 60],
    "ROI_rect_size": [120,120],

    "normalisation": "div",
    
    "single_pixel_check": [15,15],
    "area_1_centre": [7, 25],
    "area_1_halfsize": 3, 
    "area_2_centre": [20, 20],
    "area_2_halfsize": 3,
    "area_3_centre": [25, 8],
    "area_3_halfsize": 3,

    "fit_backend": "scipy",
    "fit_backend_comparison": ["gpufit"],
    "fit_pixels": True, 
    "force_fit": True, 
    "auto_match_prev_ROI_options": False,
    
    "use_ROI_avg_fit_res_for_all_pixels": True,
    
    "scipy_sub_threads": 2, 

    "fit_functions":  {"linear": 1, "lorentzian": 8},

    "pos_guess": [2610, 2708, 2805, 2879, 2953, 3020, 3092, 3145],
    "pos_range": 25,

    "amp_guess": -0.0015,
    "amp_bounds": [-0.0100, -0.0003],

    "fwhm_guess": 9,
    "fwhm_bounds": [5, 20],

    "c_guess": 1,
    "c_bounds": [0.9995, 1.0005],

    "m_guess": 0,
    "m_bounds": [-1e-6, 1e-6],
    
    "scramble_pixels": True
}

In [4]:
# QDMPy_path = data_loading.DIR_PATH
# options = data_loading.load_options(path=QDMPy_path / "options/test_options_odmr.json", check_for_prev_result=True)
options = data_loading.load_options(options_dict=options_dict, check_for_prev_result=True)
fit_plots.set_mpl_rcparams(options)

In [5]:
raw_data, prelim_sweep_list = data_loading.load_raw_and_sweep(options)

In [6]:
PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list = data_loading.reshape_dataset(
        options, raw_data, prelim_sweep_list
    )

In [7]:
fig1 = fit_plots.plot_ROI_PL_image(options, PL_image)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
AOIs = data_loading.define_AOIs(options)

In [9]:
fig2 = fit_plots.plot_AOI_PL_images(options, PL_image_ROI, AOIs)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
aoi_spectra_plot = fit_plots.plot_AOI_spectra(options, AOIs, sig, ref, sweep_list)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit AOI, ROI averages, single pixel

In [11]:
fit_model = fit_interface.define_fit_model(options)

C:\src\QDMPy\QDMPy\fit_scipy.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # auto detects jupyter


In [12]:
backend_ROI_results_lst = fit_interface.fit_ROI_avg(options, sig_norm, sweep_list, fit_model)

In [13]:
fig3 = fit_plots.plot_ROI_avg_fits(options, backend_ROI_results_lst)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
fit_result_collection_lst = fit_interface.fit_AOIs(options, sig_norm, single_pixel_pl, sweep_list, fit_model, AOIs, backend_ROI_results_lst)

In [15]:
AOI_spectra_fit_fig = fit_plots.plot_AOI_spectra_fit(options, sig, ref, sweep_list, AOIs, fit_result_collection_lst, backend_ROI_results_lst, fit_model)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit pixels

In [16]:
if (options["force_fit"] or not options["found_prev_result"]) and options["fit_pixels"]:
    pixel_fit_params = fit_interface.fit_pixels(options, sig_norm, sweep_list, fit_model, backend_ROI_results_lst[0])
elif options["found_prev_result"]:
    pixel_fit_params = fit_interface.load_prev_fit_results(options)
else:
    pixel_fit_params = None  # not fitting pixels, this stops plotting (e.g. via plot_param_images) from erroring

  0%|          | 0/16384 [00:00<?, ? PX/s]

# Plot (fit) parameters

In [17]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "c")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "m")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "pos")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "amp")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "fwhm")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
data_loading.save_options(options)

In [23]:
options

{'base_dir': 'C:\\src\\QDMPY_test_image\\',
 'filepath': 'C:\\src\\QDMPY_test_image\\ODMR - Pulsed with IR_83',
 'raw_pixel_size': 2.44140625e-07,
 'ignore_ref': False,
 'additional_bins': 4,
 'old_binning_convention': False,
 'system_name': 'Zyla',
 'ROI': 'Full',
 'ROI_halfsize': 1,
 'ROI_centre': [60, 60],
 'ROI_rect_size': [120, 120],
 'remove_start_sweep': 1,
 'remove_end_sweep': 0,
 'normalisation': 'div',
 'single_pixel_check': [15, 15],
 'area_1_centre': [7, 25, 7, 25, 7, 25, 7, 25],
 'area_1_halfsize': 3,
 'area_2_centre': [20, 20, 20, 20, 20, 20, 20, 20],
 'area_2_halfsize': 3,
 'area_3_centre': [25, 8, 25, 8, 25, 8, 25, 8],
 'area_3_halfsize': 3,
 'area_4_centre': None,
 'area_4_halfsize': None,
 'area_5_centre': None,
 'area_5_halfsize': None,
 'fit_backend': 'scipy',
 'fit_backend_comparison': ['gpufit', 'scipy'],
 'force_fit': True,
 'fit_pixels': True,
 'auto_match_prev_ROI_options': False,
 'scramble_pixels': True,
 'use_ROI_avg_fit_res_for_all_pixels': True,
 'fit_func